### imports

In [11]:
import pandas as pd
df = pd.read_csv(r"http://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv")

In [12]:
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


### basic preprocessing

In [13]:
import numpy as np
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Text preprocessing functions
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

def removing_numbers(text):
    return ''.join([char for char in text if not char.isdigit()])

def lower_case(text):
    return " ".join([word.lower() for word in text.split()])

def removing_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    return re.sub('\s+', ' ', text).strip()

def removing_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_small_sentences(df):
    df['content'] = df['content'].apply(lambda x: np.nan if len(str(x).split()) < 3 else x)
    return df

def normalize_text(df):
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        df = remove_small_sentences(df)
        return df.dropna(subset=['content'])
    except Exception as e:
        print(e)
        return df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iampr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\iampr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
df = normalize_text(df)

In [15]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [16]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

In [17]:
df.sample(5)

,tweet_id,sentiment,content
7899,1961922392,0,skype call billie webcam dont work
22376,1694419708,1,drchino yay joy department
24125,1694769245,1,happy star war day everyone excuse drinkies eh
2462,1957549892,0,fixed favourite heel fell wear
26567,1695443815,1,diannemr like that


### Experiment 1

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 10)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

#### logging experiment on mlflow


In [20]:
import dagshub
import mlflow

dagshub.init(repo_owner='iamprashantjain', repo_name='Emotion-Detection-MLOps', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/iamprashantjain/Emotion-Detection-MLOps.mlflow")
mlflow.set_experiment("baseline_model")

Initialized MLflow to track repo "iamprashantjain/Emotion-Detection-MLOps"

Repository iamprashantjain/Emotion-Detection-MLOps initialized!

2025/07/20 23:20:04 INFO mlflow.tracking.fluent: Experiment with name 'baseline_model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/a5492073f5de4e03875e2572ddb4e014', creation_time=1753033805239, experiment_id='3', last_update_time=1753033805239, lifecycle_stage='active', name='baseline_model', tags={}>

#### run mlflow experiment

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

with mlflow.start_run():
    # Log preprocessing params
    mlflow.log_param("vectorizer", "BOW")
    mlflow.log_param("num_features", 10)
    mlflow.log_param("test_size", 0.2)

    # Model building & training
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Log model params
    mlflow.log_param("model", "logistic regression")

    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Optionally log the model
    mlflow.sklearn.log_model(model, "model")

    # Log the notebook
    import os
    notebook_path = "experiments.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace{notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Print results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

i:\CampusX_DS\campusx_dsmp2\9. MLOps revisited\tutorial\Emotion-Detection-MLOps\venv\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


Accuracy: 0.5667
Precision: 0.6487
Recall: 0.2812
F1 Score: 0.3924


### Experiment 2
apply mulitple algorithm with both vectorizer (bow & tfidf) to find which combination is giving best accuracy

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier


mlflow.set_experiment("BOW V/S TFIDF")

2025/07/20 23:20:56 INFO mlflow.tracking.fluent: Experiment with name 'BOW V/S TFIDF' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7ece5e3805584630972fe0dd04783876', creation_time=1753033856903, experiment_id='4', last_update_time=1753033856903, lifecycle_stage='active', name='BOW V/S TFIDF', tags={}>

In [23]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=200),
    "SVC": SVC(probability=True),
    "NaiveBayes": MultinomialNB(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}
vectorizers = {
    "BOW": CountVectorizer(max_features=10),
    "TFIDF": TfidfVectorizer(max_features=10)
}

In [26]:
with mlflow.start_run(run_name = "All Combination Experiment") as parent_run:
    for model_name, model in models.items():
        for vectorizer_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{model_name}_{vectorizer_name}", nested=True):

                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                # Preprocessing parameters
                mlflow.log_param("model_name", model_name)
                mlflow.log_param("vectorizer", vectorizer_name)
                mlflow.log_param("num_features", 10)
                mlflow.log_param("test_size", 0.2)

                # Model-specific parameter logging
                if model_name == "LogisticRegression":
                    mlflow.log_param("solver", model.solver if hasattr(model, "solver") else "lbfgs")
                    mlflow.log_param("max_iter", model.max_iter)
                elif model_name == "SVC":
                    mlflow.log_param("kernel", model.kernel if hasattr(model, "kernel") else "rbf")
                    mlflow.log_param("C", model.C)
                elif model_name == "NaiveBayes":
                    mlflow.log_param("alpha", model.alpha)
                elif model_name == "XGBoost":
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)

                # Fit model
                model.fit(X_train, y_train)

                # Predictions
                y_pred = model.predict(X_test)

                # Evaluation metrics
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=0)
                recall = recall_score(y_test, y_pred, zero_division=0)
                f1 = f1_score(y_test, y_pred, zero_division=0)

                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)

                # Log the model
                mlflow.sklearn.log_model(model, artifact_path="model")
                
                # Log the notebook
                import os
                notebook_path = "experiments.ipynb"
                os.system(f"jupyter nbconvert --to notebook --execute --inplace{notebook_path}")
                mlflow.log_artifact(notebook_path)
                
                print(f"[{model_name} + {vectorizer_name}] F1 Score: {f1:.4f}")

[LogisticRegression + BOW] F1 Score: 0.3924
[LogisticRegression + TFIDF] F1 Score: 0.3955
[SVC + BOW] F1 Score: 0.3827
[SVC + TFIDF] F1 Score: 0.3787
[NaiveBayes + BOW] F1 Score: 0.6406
[NaiveBayes + TFIDF] F1 Score: 0.6395


i:\CampusX_DS\campusx_dsmp2\9. MLOps revisited\tutorial\Emotion-Detection-MLOps\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[XGBoost + BOW] F1 Score: 0.3764


i:\CampusX_DS\campusx_dsmp2\9. MLOps revisited\tutorial\Emotion-Detection-MLOps\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [23:36:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[XGBoost + TFIDF] F1 Score: 0.3744


### Experiment 3 - Hyper Parameter Tuning

In [29]:
#we will take - bow + lr
#find best params of best model & vectorizer combination

In [31]:
from sklearn.model_selection import GridSearchCV, train_test_split
import mlflow.sklearn

# Hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],  # 'liblinear' supports l1 and l2
    'max_iter': [100, 200]
}

# MLflow Tracking
mlflow.set_experiment("BOW_LR_Hyperparameter_tuning")

with mlflow.start_run(run_name="Parent_Run") as parent_run:
    
    # Hyperparameter Tuning
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each param combination as a child run
    for idx, params in enumerate(grid_search.cv_results_['params']):
        mean_score = grid_search.cv_results_['mean_test_score'][idx]
        std_score = grid_search.cv_results_['std_test_score'][idx]

        with mlflow.start_run(run_name=f"LR_{params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, zero_division=0)
            recall = recall_score(y_test, y_pred, zero_division=0)
            f1 = f1_score(y_test, y_pred, zero_division=0)

            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_f1", mean_score)
            mlflow.log_metric("std_cv_f1", std_score)
            mlflow.log_metric("test_accuracy", accuracy)
            mlflow.log_metric("test_precision", precision)
            mlflow.log_metric("test_recall", recall)
            mlflow.log_metric("test_f1", f1)

    # Log best model details in parent run
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_cv_f1", grid_search.best_score_)

    # Evaluate best model on test set
    best_pred = best_model.predict(X_test)
    test_f1 = f1_score(y_test, best_pred, zero_division=0)
    mlflow.log_metric("best_test_f1", test_f1)

    mlflow.sklearn.log_model(best_model, artifact_path="best_model")

    print(f"Best Parameters: {best_params}")
    print(f"Test F1 Score with Best Model: {test_f1:.4f}")

Best Parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Test F1 Score with Best Model: 0.6434
